In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import combinations

In [2]:
data_path = 'data/'
train = pd.read_csv(data_path+'sensor_train.csv')
test = pd.read_csv(data_path+'sensor_test.csv')

## 数据分析

In [4]:
train.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0


In [10]:
train['abs_acc_x']=abs(train['acc_x'])

In [11]:
train.describe()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,abs_acc_x
count,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000
mean,3647.035972,2500.952026,0.009635,-0.063114,0.024548,0.965656,2.730190,7.825065,8.535035,0.307232
std,2103.253140,1442.307956,0.619285,0.537614,0.717009,3.295531,2.978134,2.653261,5.395264,0.537787
min,0.000000,0.000000,-14.700000,-14.700000,-12.600000,-19.800000,-16.900000,-8.900000,0.000000,0.000000
25%,1822.000000,1252.000000,-0.100000,-0.100000,-0.200000,-0.700000,0.400000,7.300000,4.000000,0.000000
50%,3646.000000,2501.000000,0.000000,0.000000,0.000000,0.200000,2.700000,8.700000,8.000000,0.100000
75%,5466.000000,3749.000000,0.100000,0.100000,0.200000,2.000000,4.900000,9.400000,13.000000,0.400000
max,7291.000000,4999.000000,34.500000,13.400000,28.200000,30.200000,14.400000,38.100000,18.000000,34.500000


In [8]:
def statics(data):
    stats = []
    for col in data.columns:
        stats.append((col, data[col].nunique(), data[col].isnull().sum() * 100 / data.shape[0],
                      data[col].value_counts(normalize=True, dropna=False).values[0] * 100, data[col].dtype))

    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Percentage_of_missing_values',
                                            'Percentage_of_values_in_the_biggest category', 'type'])
    stats_df.sort_values('Percentage_of_missing_values', ascending=False, inplace=True)
    return stats_df

In [9]:
train_stats_df=statics(train)
train_stats_df

,Feature,Unique_values,Percentage_of_missing_values,Percentage_of_values_in_the_biggest category,type
0,fragment_id,7292,0.0,0.014341,int64
1,time_point,5000,0.0,0.033148,int64
2,acc_x,212,0.0,27.568948,float64
3,acc_y,159,0.0,34.423393,float64
4,acc_z,218,0.0,19.457447,float64
5,acc_xg,300,0.0,3.593905,float64
6,acc_yg,272,0.0,2.087648,float64
7,acc_zg,284,0.0,4.472457,float64
8,behavior_id,19,0.0,10.260980,int64


## LSTM数据构造

In [3]:
label = 'behavior_id'
test['fragment_id'] += 10000

data = pd.concat([train, test], sort=False)
base_fea=['time_point','acc_x','acc_y','acc_z','acc_xg','acc_yg','acc_zg']
for col in base_fea:
    data['{}_rank'.format(col)] = data[col].rank()


# print("加减乘除...")
# for a, b in tqdm(combinations(base_fea, 2)):
#     data[str(a) + '+' + str(b)] = data[a].astype(float) + data[b].astype(float)
#     data[str(a) + '-' + str(b)] = data[a].astype(float) - data[b].astype(float)
#     data[str(a) + '*' + str(b)] = data[a].astype(float) * data[b].astype(float)
# #     data[str(a) + '/' + str(b)] = data[a].astype(float) / (data[b].astype(float) + 1)
    # df[str(a) + '/log' + str(b)] = df[a].astype(float) / np.log1p(df[b].astype(float))
# # 类别特征计数
# print("类别特征计数...")
# for i in tqdm(base_fea):
#     data['{}_count'.format(i)] = data.groupby(['{}'.format(i)])['fragment_id'].transform('count')

# # 类别特征nunqiue
# print("类别特征nunqiue...")
# for i in tqdm(base_fea):
#     for j in base_fea:
#         if i != j:
#             base_fea['nuni_{0}_{1}'.format(i, j)] = base_fea[i].map(base_fea.groupby(i)[j].nunique())

def get_cvr_fea(df):
    cat_list = ['time_point','acc_x','acc_y','acc_z','acc_xg','acc_yg','acc_zg']
    print("cat_list", cat_list)

    # 类别特征五折转化率特征
    print("转化率特征....")
    df['ID'] = df.index
    df['fold'] = df['ID'] % 5
    df.loc[df.behavior_id.isnull(), 'fold'] = 5
    target_feat = []
    for i in tqdm(cat_list):
        target_feat.extend([i + '_mean_last_1'])
        df[i + '_mean_last_1'] = None
        for fold in range(6):
            df.loc[df['fold'] == fold, i + '_mean_last_1'] = df[df['fold'] == fold][i].map(
                df[(df['fold'] != fold) & (df['fold'] != 5)].groupby(i)['behavior_id'].mean()
            )
        df[i + '_mean_last_1'] = df[i + '_mean_last_1'].astype(float)

    return df
data = get_cvr_fea(data)

data['acc'] = (data['acc_x'] ** 2 + data['acc_y'] ** 2 + data['acc_z'] ** 2) ** 0.5
data['accg'] = (data['acc_xg'] ** 2 + data['acc_yg'] ** 2 + data['acc_zg'] ** 2) ** 0.5
for j in tqdm(base_fea):
    data['fragment_id_id_{}_mean'.format(j)] = data.groupby(['fragment_id'])[j].transform('mean')
    data['fragment_id_id_{}_median'.format(j)] = data.groupby(['fragment_id'])[j].transform('median')
    data['fragment_id_{}_max'.format(j)] = data.groupby(['fragment_id'])[j].transform('max')
    data['fragment_id_id_{}_min'.format(j)] = data.groupby(['fragment_id'])[j].transform('min')

  0%|                                                                                                                                      | 0/7 [00:00<?, ?it/s]

cat_list ['time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg', 'acc_yg', 'acc_zg']
转化率特征....


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.74it/s]


In [4]:
drop_feat = []
used_feat = [f for f in data.columns if f not in (['fragment_id', label] + drop_feat)]
used_feat

['time_point',
 'acc_x',
 'acc_y',
 'acc_z',
 'acc_xg',
 'acc_yg',
 'acc_zg',
 'time_point_rank',
 'acc_x_rank',
 'acc_y_rank',
 'acc_z_rank',
 'acc_xg_rank',
 'acc_yg_rank',
 'acc_zg_rank',
 'ID',
 'fold',
 'time_point_mean_last_1',
 'acc_x_mean_last_1',
 'acc_y_mean_last_1',
 'acc_z_mean_last_1',
 'acc_xg_mean_last_1',
 'acc_yg_mean_last_1',
 'acc_zg_mean_last_1',
 'acc',
 'accg',
 'fragment_id_id_time_point_mean',
 'fragment_id_id_time_point_median',
 'fragment_id_time_point_max',
 'fragment_id_id_time_point_min',
 'fragment_id_id_acc_x_mean',
 'fragment_id_id_acc_x_median',
 'fragment_id_acc_x_max',
 'fragment_id_id_acc_x_min',
 'fragment_id_id_acc_y_mean',
 'fragment_id_id_acc_y_median',
 'fragment_id_acc_y_max',
 'fragment_id_id_acc_y_min',
 'fragment_id_id_acc_z_mean',
 'fragment_id_id_acc_z_median',
 'fragment_id_acc_z_max',
 'fragment_id_id_acc_z_min',
 'fragment_id_id_acc_xg_mean',
 'fragment_id_id_acc_xg_median',
 'fragment_id_acc_xg_max',
 'fragment_id_id_acc_xg_min',
 'fra

In [5]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
data[used_feat] = min_max_scaler.fit_transform(data[used_feat])

data.shape

(855541, 55)

In [6]:

train = data[data[label].isna()==False].reset_index(drop=True)
test = data[data[label].isna()==True].reset_index(drop=True)


In [7]:
train.shape

(425359, 55)

In [8]:
test.shape

(430182, 55)

In [9]:
# 检查是否存在NAN值

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425359 entries, 0 to 425358
Data columns (total 55 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   fragment_id                       425359 non-null  int64  
 1   time_point                        425359 non-null  float64
 2   acc_x                             425359 non-null  float64
 3   acc_y                             425359 non-null  float64
 4   acc_z                             425359 non-null  float64
 5   acc_xg                            425359 non-null  float64
 6   acc_yg                            425359 non-null  float64
 7   acc_zg                            425359 non-null  float64
 8   behavior_id                       425359 non-null  float64
 9   time_point_rank                   425359 non-null  float64
 10  acc_x_rank                        425359 non-null  float64
 11  acc_y_rank                        425359 non-null  f

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430182 entries, 0 to 430181
Data columns (total 55 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   fragment_id                       430182 non-null  int64  
 1   time_point                        430182 non-null  float64
 2   acc_x                             430182 non-null  float64
 3   acc_y                             430182 non-null  float64
 4   acc_z                             430182 non-null  float64
 5   acc_xg                            430182 non-null  float64
 6   acc_yg                            430182 non-null  float64
 7   acc_zg                            430182 non-null  float64
 8   behavior_id                       0 non-null       float64
 9   time_point_rank                   430182 non-null  float64
 10  acc_x_rank                        430182 non-null  float64
 11  acc_y_rank                        430182 non-null  f

In [12]:
# fragment_id统计
train.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,time_point_rank,...,fragment_id_acc_xg_max,fragment_id_id_acc_xg_min,fragment_id_id_acc_yg_mean,fragment_id_id_acc_yg_median,fragment_id_acc_yg_max,fragment_id_id_acc_yg_min,fragment_id_id_acc_zg_mean,fragment_id_id_acc_zg_median,fragment_id_acc_zg_max,fragment_id_id_acc_zg_min
0,0,0.005401,0.324111,0.321429,0.199739,0.439394,0.716332,0.285528,0.0,0.003919,...,0.232941,0.714689,0.722787,0.724324,0.555556,0.824503,0.862863,0.895833,0.19076,0.886463
1,0,0.021604,0.320158,0.328125,0.193211,0.436027,0.722063,0.280313,0.0,0.017101,...,0.232941,0.714689,0.722787,0.724324,0.555556,0.824503,0.862863,0.895833,0.19076,0.886463
2,0,0.039608,0.320158,0.328125,0.202350,0.444444,0.719198,0.288136,0.0,0.035000,...,0.232941,0.714689,0.722787,0.724324,0.555556,0.824503,0.862863,0.895833,0.19076,0.886463
3,0,0.059412,0.320158,0.325893,0.191906,0.442761,0.722063,0.264668,0.0,0.054702,...,0.232941,0.714689,0.722787,0.724324,0.555556,0.824503,0.862863,0.895833,0.19076,0.886463
4,0,0.077616,0.320158,0.332589,0.206266,0.444444,0.722063,0.286832,0.0,0.073102,...,0.232941,0.714689,0.722787,0.724324,0.555556,0.824503,0.862863,0.895833,0.19076,0.886463


In [13]:
len(train['fragment_id'].unique()),len(test['fragment_id'].unique()),

(7292, 7500)

In [14]:
## 读取和理解数据

In [15]:
train_sequences=list()
base_fea=used_feat

for index,group in train.groupby(by='fragment_id'):
    train_sequences.append(group[base_fea].values)
train_sequences[0]
y_train=train.drop_duplicates(subset=['fragment_id']).reset_index(drop=True)['behavior_id'].values

In [16]:
len_sequences=[]
for one_seq in train_sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    7292.000000
mean       58.332282
std         1.668028
min        50.000000
25%        57.000000
50%        58.000000
75%        60.000000
max        61.000000
dtype: float64

## 训练集

In [17]:
# padding the sequence with the values in last row to max length
to_pad=61
train_new_seq=[]
for one_seq in train_sequences:
    len_one_seq=len(one_seq)
    last_val=one_seq[-1]
    n=to_pad-len_one_seq
    to_concat=np.repeat(last_val,n).reshape(len(used_feat),n).transpose()
    new_one_seq=np.concatenate([one_seq,to_concat])
    train_new_seq.append(new_one_seq)

train_final_seq=np.stack(train_new_seq)
# final_seq.shape (314, 129, 4)
train_final_seq.shape
# 进行截断

(7292, 61, 53)

In [18]:
from keras.preprocessing import sequence
seq_len=60
train_final_seq=sequence.pad_sequences(train_final_seq,maxlen=seq_len,padding='post',
                                dtype='float',truncating='post')
train_final_seq.shape

(7292, 60, 53)

In [19]:
train_final_seq

array([[[0.00540108, 0.32411067, 0.32142857, ..., 0.89583333,
         0.19076006, 0.88646288],
        [0.02160432, 0.3201581 , 0.328125  , ..., 0.89583333,
         0.19076006, 0.88646288],
        [0.03960792, 0.3201581 , 0.328125  , ..., 0.89583333,
         0.19076006, 0.88646288],
        ...,
        [0.99039808, 0.32411067, 0.33035714, ..., 0.89583333,
         0.19076006, 0.88646288],
        [0.99039808, 0.32411067, 0.33035714, ..., 0.89583333,
         0.19076006, 0.88646288],
        [0.99039808, 0.32411067, 0.33035714, ..., 0.89583333,
         0.19076006, 0.88646288]],

       [[0.01160232, 0.31620553, 0.328125  , ..., 0.89583333,
         0.19225037, 0.92576419],
        [0.02920584, 0.3201581 , 0.328125  , ..., 0.89583333,
         0.19225037, 0.92576419],
        [0.04660932, 0.31818182, 0.33035714, ..., 0.89583333,
         0.19225037, 0.92576419],
        ...,
        [0.99259852, 0.3201581 , 0.328125  , ..., 0.89583333,
         0.19225037, 0.92576419],
        [0.9

## 测试集

In [20]:
test_sequences=list()
base_fea=used_feat

for index,group in test.groupby(by='fragment_id'):
    test_sequences.append(group[base_fea].values)

# padding the sequence with the values in last row to max length
to_pad=61
test_new_seq=[]
for one_seq in test_sequences:
    len_one_seq=len(one_seq)
    last_val=one_seq[-1]
    n=to_pad-len_one_seq
    to_concat=np.repeat(last_val,n).reshape(len(used_feat),n).transpose()
    new_one_seq=np.concatenate([one_seq,to_concat])
    test_new_seq.append(new_one_seq)

test_final_seq=np.stack(test_new_seq)
# final_seq.shape (314, 129, 4)
test_final_seq.shape


# 进行截断
from keras.preprocessing import sequence
seq_len=60
test_final_seq=sequence.pad_sequences(test_final_seq,maxlen=seq_len,padding='post',
                                dtype='float',truncating='post')
test_final_seq.shape

(7500, 60, 53)

In [21]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)

In [22]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

## 打乱数据集

In [23]:
from sklearn.utils import shuffle
X,y=shuffle(train_final_seq,y_train,random_state=42)


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.1, random_state=42)

In [25]:
y_valid.shape

(730, 19)

## 模型预训练


In [26]:
import matplotlib.pyplot as plt
from os import listdir
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import *
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization


In [27]:
input_layer=Input(shape=(seq_len,len(used_feat)),name="input_layer")
lstm_layer=LSTM(256)(input_layer)
# bp=BatchNormalization()(lstm_layer)
# dense=Dense(64)(bp)
pred=Dense(19,activation='softmax')(lstm_layer)
model=Model(input_layer,pred)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 60, 53)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               317440    
_________________________________________________________________
dense (Dense)                (None, 19)                4883      
Total params: 322,323
Trainable params: 322,323
Non-trainable params: 0
_________________________________________________________________


In [33]:
adam=Adam(lr=0.001)
checkpoint=ModelCheckpoint('data/best_model.pkl',monitor='val_accuracy',save_best_only=True,verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)
model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint,early_stop],
    validation_data=(X_valid, y_valid)
)

Epoch 1/100
205/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0338
Epoch 00001: val_accuracy improved from -inf to 0.02603, saving model to data/best_model.pkl
INFO:tensorflow:Assets written to: data/best_model.pkl\assets
206/206 [==============================] - 4s 21ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 2/100
203/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0340
Epoch 00002: val_accuracy did not improve from 0.02603
206/206 [==============================] - 1s 7ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 3/100
205/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0337
Epoch 00003: val_accuracy did not improve from 0.02603
206/206 [==============================] - 1s 7ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 4/100
203/206 [====================

Epoch 29/100
202/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0340
Epoch 00029: val_accuracy did not improve from 0.02603
206/206 [==============================] - 1s 7ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 30/100
203/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0339
Epoch 00030: val_accuracy did not improve from 0.02603
206/206 [==============================] - 1s 6ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 31/100
203/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0340
Epoch 00031: val_accuracy did not improve from 0.02603
206/206 [==============================] - 1s 6ms/step - loss: 14.4466 - accuracy: 0.0338 - val_loss: 14.4467 - val_accuracy: 0.0260
Epoch 32/100
204/206 [============================>.] - ETA: 0s - loss: 14.4466 - accuracy: 0.0337
Epoch 00032: val_accuracy did not improv

In [ ]:
# 加载以前保存的权重
model=tf.keras.models.load_model('./data/best_model.pkl')
from sklearn.metrics import accuracy_score
test_preds=model.predict(test_final_seq)

In [ ]:
label=np.argmax(test_preds,axis=1)
pd.Series(label).value_counts()

In [ ]:
sub = pd.read_csv(data_path+'提交结果示例.csv')
sub['behavior_id'] = label
sub.to_csv('result/lstm.csv', index=False)